### Column {.tabset}


In [ ]:
#| label: MAP1
#| echo: false
#| warning: false
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Loading data
data = pd.read_csv("data/owid-energy-data.csv")
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merging world shapefile with energy data
world = world.merge(data, how='left', left_on='iso_a3', right_on='iso_code')

# Droping NaN values from columns used in calculations
columns_to_drop_na = ['solar_consumption', 'wind_consumption', 'hydro_consumption', 'other_renewable_consumption', 'energy_per_capita']

# Filtering data from 2000 to 2023
world = world[(world['year'] >= 2000) & (world['year'] <= 2023)]

# Grouping by year and country, summing renewable energy consumption and energy per capita
world = world.groupby(['iso_a3', 'year', 'name']).agg({
    'solar_consumption': 'sum',
    'wind_consumption': 'sum',
    'hydro_consumption': 'sum',
    'other_renewable_consumption': 'sum',
    'energy_per_capita': 'mean'
}).reset_index()

# Calculating total renewable energy consumption
world['total_renewable_consumption'] = world['solar_consumption'] + world['wind_consumption'] + world['hydro_consumption'] + world['other_renewable_consumption']

# Calculate renewable energy share
world['renewable_energy_share'] = (world['total_renewable_consumption'] / world['energy_per_capita']) * 100

# Setting year column as date
world['year'] = pd.to_datetime(world['year'], format='%Y')

# Plotting the animated map
fig = px.choropleth(world, 
                    locations='iso_a3',
                    color='renewable_energy_share',
                    hover_name='name',
                    hover_data={'iso_a3': False, 'renewable_energy_share': True},
                    animation_frame=world['year'].dt.year,  
                    range_color=(0, 7),  
                    projection='natural earth',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title='Share of Renewable Energy Consumption (%)')

# Setting x-axis format to display only the year
fig.update_xaxes(dtick='M1', tickformat='%Y')

fig.show()

In [ ]:
#| label: BAR5
#| echo: false
#| warning: false
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merging world shapefile with energy data
world = world.merge(data, how='left', left_on='iso_a3', right_on='iso_code')

# Droping NaN values from columns used in calculations
columns_to_drop_na = ['solar_consumption', 'wind_consumption', 'hydro_consumption', 'other_renewable_consumption', 'energy_per_capita']

# Filtering data from 2000 to 2023
world = world[(world['year'] >= 2000) & (world['year'] <= 2023)]

# Grouping by year and country, summing renewable energy consumption and energy per capita
world = world.groupby(['iso_a3', 'year', 'name']).agg({
    'solar_consumption': 'sum',
    'wind_consumption': 'sum',
    'hydro_consumption': 'sum',
    'other_renewable_consumption': 'sum',
    'energy_per_capita': 'mean'
}).reset_index()

# Calculating total renewable energy consumption
world['total_renewable_consumption'] = world['solar_consumption'] + world['wind_consumption'] + world['hydro_consumption'] + world['other_renewable_consumption']

# Calculate renewable energy share
world['renewable_energy_share'] = (world['total_renewable_consumption'] / world['energy_per_capita']) * 100

# Setting year column as date
world['year'] = pd.to_datetime(world['year'], format='%Y')

# Calculate mean renewable energy share by country
country_stats = world.groupby('name')['renewable_energy_share'].mean().reset_index()

# Identify top 10 countries with the highest mean renewable energy share
top_10_countries = country_stats.nlargest(10, 'renewable_energy_share')['name'].tolist()

# Filter data for top 10 countries
top_10_world = world[world['name'].isin(top_10_countries)]

# Sort data by renewable energy share
top_10_world = top_10_world.sort_values(by='renewable_energy_share', ascending=False)

# Plotting the animated bar plot with Plotly Graph Objects
fig = go.Figure()

# Create bar traces for each year
for year, df in top_10_world.groupby('year'):
    fig.add_trace(go.Bar(
        x=df['name'],
        y=df['renewable_energy_share'],
        name=str(year.year),
        hoverinfo='x+y',
        hovertemplate='<b>%{x}</b><br>Renewable Energy Share: %{y:.2f}%<extra></extra>',
        visible=False if year != top_10_world['year'].min() else True,
        marker=dict(color='rgba(50, 171, 96, 0.6)')
    ))

# Add play button and slider
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=[dict(label="Play",
                      method="animate",
                      args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "transition": {"duration": 300, "easing": "quadratic-in-out"}}]
        )]
    )],
    title='Top 10 Countries with Highest Mean Renewable Energy Share',
    xaxis=dict(title='Country'),
    yaxis=dict(title='Mean Renewable Energy Share (%)', range=[0, 20])
)

# Set initial layout
fig.update_layout(showlegend=True,)

# Create frames for each year
frames = [go.Frame(
    data=[go.Bar(
        x=df['name'],
        y=df['renewable_energy_share'],
        name=str(year.year),
        hoverinfo='x+y',
        hovertemplate='<b>%{x}</b><br>Renewable Energy Share: %{y:.2f}%<extra></extra>',
        marker=dict(color='rgba(50, 171, 96, 0.6)')
    )],
    name=str(year.year)
) for year, df in top_10_world.groupby('year')]

# Add frames to the figure
fig.frames = frames

fig.show()

In [ ]:
#| label: data_wrang_density
#| warning: false
#| echo: false

## loading the data set
import pandas as pd
energy = pd.read_csv('data/owid-energy-data.csv') 

## performing data wrangling operations on the data set to make it suitable for the analysis

## selecting the necessary columns for this analysis
energy2 = energy[['year','country','iso_code', 'biofuel_consumption', 'coal_consumption', 'fossil_fuel_consumption', 'gas_consumption', 'hydro_consumption', 'nuclear_consumption', 'oil_consumption', 'other_renewable_consumption', 'renewables_consumption', 'solar_consumption', 'wind_consumption']]


### removing the rows with only missing values in it with the exception of the year column
energy2_cleaned = energy2.dropna(subset = energy2.columns.difference(['year']), how = 'all')


### we remove the rows in the country column that are not countries but rather regions or continents

### loading the data set for the country code
country_codes = pd.read_csv('data/country_codes.csv')

# filtering the energy data set by removing regions which are not countries from the country column in the data set

## first we select the valid country codes
valid_country_codes = set(country_codes['Alpha-3 code'])


## filtering the country column in the data set
energy_filtered = energy2_cleaned[energy2_cleaned['iso_code'].isin(valid_country_codes)]


###removing all rows with NA values except the iso_codes, year and country columns in the data set
energy_wrangled = energy_filtered.dropna(subset = energy_filtered.columns.difference(['year', 'iso_code', 'country']), how = 'all')


### removing the values with na values in the data set
energy_wrangled = energy_wrangled.fillna(0)



# Select columns to check for 0 values
columns_to_check = energy_wrangled.columns.difference(['year', 'iso_code', 'country'])

# Filter rows with 0 values in all columns except 'year', 'country', and 'iso_code'
energy_wrangled = energy_wrangled[(energy_wrangled[columns_to_check] != 0).any(axis = 1)]

### The next step is feature engineering where new columns in the data set are created

## note that there is a column for the renewable energy consumption total
### creating a column for the total consumption of non-renewable energy
energy_wrangled['non_renewables_consumption'] = energy_wrangled['coal_consumption'] + energy_wrangled['fossil_fuel_consumption'] + energy_wrangled['gas_consumption'] +  energy_wrangled['nuclear_consumption'] + energy_wrangled['oil_consumption']

### rounding the rows in the newly column to 3 decimal places
energy_wrangled['non_renewables_consumption'] = energy_wrangled['non_renewables_consumption'].round(3)

### a new column for the total consumption is created 
energy_wrangled['total_consumption'] = energy_wrangled['non_renewables_consumption'] + energy_wrangled['renewables_consumption']

#### making sure that the total consumption is in two decimal places 
energy_wrangled['total_consumption'] = energy_wrangled['total_consumption'].round(3)




### the next step is to feature engineer and group the data into continents

### generating a function that will group the countries into their respective continents
def group_countries_by_continent(iso_code):
    continents = {
        'Africa': ['DZA', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CPV', 'CMR', 'CAF', 'TCD', 'COM', 'COG', 'COD', 'DJI', 'EGY', 'GNQ', 'ERI', 'SWZ', 'ETH', 'GAB', 'GMB', 'GHA', 'GIN', 'GNB', 'CIV', 'KEN', 'LSO', 'LBR', 'LBY', 'MDG', 'MWI', 'MLI', 'MRT', 'MUS', 'MAR', 'MOZ', 'NAM', 'NER', 'NGA', 'RWA', 'STP', 'SEN', 'SYC', 'SLE', 'SOM', 'ZAF', 'SSD', 'SDN', 'TZA', 'TGO', 'TUN', 'UGA', 'ZMB', 'ZWE'],
        'Asia': ['AFG', 'ARM', 'AZE', 'BHR', 'BGD', 'BTN', 'BRN', 'KHM', 'CHN', 'CYP', 'GEO', 'IND', 'IDN', 'IRN', 'IRQ', 'ISR', 'JPN', 'JOR', 'KAZ', 'KWT', 'KGZ', 'LAO', 'LBN', 'MYS', 'MDV', 'MNG', 'MMR', 'NPL', 'PRK', 'OMN', 'PAK', 'PSE', 'PHL', 'QAT', 'SAU', 'SGP', 'KOR', 'LKA', 'SYR', 'TWN', 'TJK', 'THA', 'TLS', 'TKM', 'ARE', 'UZB', 'VNM', 'YEM', 'TUR', 'THA'],
        'Europe': ['ALB', 'AND', 'AUT', 'BLR', 'BEL', 'BIH', 'BGR', 'HRV', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ITA', 'XKX', 'LVA', 'LIE', 'LTU', 'LUX', 'MLT', 'MDA', 'MCO', 'MNE', 'NLD', 'MKD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SRB', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'UKR', 'GBR', 'VAT'],
        'North America': ['ATG', 'BHS', 'BRB', 'BLZ', 'CAN', 'CRI', 'CUB', 'DMA', 'DOM', 'SLV', 'GRL', 'GRD', 'GTM', 'HTI', 'HND', 'JAM', 'MEX', 'NIC', 'PAN', 'PRI', 'KNA', 'LCA', 'VCT', 'TTO', 'USA'],
        'South America': ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'FLK', 'GUF', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'],
        'Antarctica': ['ATA'],
        'Europe': ['ALB', 'AND', 'AUT', 'BLR', 'BEL', 'BIH', 'BGR', 'HRV', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ITA', 'XKX', 'LVA', 'LIE', 'LTU', 'LUX', 'MLT', 'MDA', 'MCO', 'MNE', 'NLD', 'MKD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SRB', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'UKR', 'GBR', 'VAT'],
        'Australia': ['AUS', 'NZL', 'PNG', 'FJI', 'SLB', 'VUT', 'NZL']
    }
    
    for continent, countries in continents.items():
        if iso_code in countries:
            return continent
    
    return None  # Return None if ISO code not found in any continent

# Apply the function to create the 'continents' column in energy_wrangled DataFrame
energy_wrangled['continents'] = energy_wrangled['iso_code'].apply(group_countries_by_continent)

### removing hong kong since it is not a country from the data set
energy_wrangled = energy_wrangled[energy_wrangled['iso_code'] != 'HKG']

### Column {.tabset}

#### Renewable
```{python}
#| label: density1_redone
#| output: false

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FuncFormatter
from IPython.display import HTML

# Extract unique years from the 'year' column
years = energy_wrangled['year'].unique()

# Function to format the x-axis labels with commas
def format_with_commas(value, pos):
    return "{:,}".format(int(value))

# Function to update the plot for the selected year
def update_plot(year):
    # Filter data for the selected year
    data_year = energy_wrangled[energy_wrangled['year'] == year]

    # Clear the previous plot
    plt.clf()

    # Plot the KDE plot for the selected year
    sns.kdeplot(data=data_year, x='renewables_consumption', hue='continents', multiple='stack', alpha=0.3, linewidth=0.2)

    # Set title
    plt.title(f'Density Plot of Renewables Consumption by Continent for Year {year}')
    # Set the x label
    plt.xlabel('Renewable Energy Consumption')
    # Set the x-axis tick formatter
    plt.gca().xaxis.set_major_formatter(FuncFormatter(format_with_commas))

# Initialize the plot with the first year
update_plot(years[0])

# Create a FuncAnimation object without controls (slider, play/pause button)
ani2 = FuncAnimation(plt.gcf(), update_plot, frames=years, interval=1000)

# Convert the animation to HTML format
html_animation2 = ani2.to_jshtml()

HTML(html_animation2)

#### Non-renewables

In [ ]:
#| label: density2_redone
#| output: false


import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FuncFormatter

# Extract unique years from the 'year' column
years = energy_wrangled['year'].unique()

# Function to format the x-axis labels with commas
def format_with_commas(value, pos):
    return "{:,}".format(int(value))

# Function to update the plot for the selected year
def update_plot(year):
    # Filter data for the selected year
    data_year = energy_wrangled[energy_wrangled['year'] == year]

    # Clear the previous plot
    plt.clf()

    # Plot the KDE plot for the selected year
    sns.kdeplot(data=data_year, x='non_renewables_consumption', hue='continents', multiple='stack', alpha=0.3, linewidth=0.2)

    # Set title
    plt.title(f'Density Plot of Non-Renewables Consumption by Continent for Year {year}')
    # Set the x label
    plt.xlabel('Non-Renewable Energy Consumption')
    # Set the x-axis tick formatter
    plt.gca().xaxis.set_major_formatter(FuncFormatter(format_with_commas))

# Initialize the plot with the first year
update_plot(years[0])

# Create a FuncAnimation object without controls (slider, play/pause button)
ani = FuncAnimation(plt.gcf(), update_plot, frames=years, interval=1000)

# Convert the animation to HTML format
html_animation = ani.to_jshtml()

# Display the HTML animation
HTML(html_animation)